# Import Modules

In [1]:
import os
import pandas as pd
import numpy.random as rnd
import warnings
import json
import sqlalchemy as sql
import datetime
import yfinance as yf
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
#from MCForecastTools import MCSimulation



%matplotlib inline

#warnings.filterwarnings("ignore")
#pd.options.display.float_format = '{:.4%}'.format

# Data Input for Universe (Multiple Asset Classes)

In [2]:
# Date range
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*5) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

# Tickers of assets
assets = ["MSFT", "TSLA", "META", "UNH", "JNJ", "JPM", "V", "LLY", "AVGO", "PG", "MA", "HD", "MRK", "NVDA", "AMZN", "BRK-B",
         "GOOG", "XOM", "CVX", "PEP", "COST", "KO", "ABBV", "ADBE", "WMT", "MCD", "CSCO", "CRM"]
assets.sort()

# Downloading data
og_data = yf.download(assets, start = start, end = end)
data = og_data.loc[:,('Adj Close', slice(None))]
data.columns = assets

Y = data[assets].pct_change().dropna()

display(start)
display(end)
display(Y)

[*********************100%***********************]  28 of 28 completed


'2018-07-09'

'2023-07-08'

,ABBV,ADBE,AMZN,AVGO,BRK-B,COST,CRM,CSCO,CVX,GOOG,...,MRK,MSFT,NVDA,PEP,PG,TSLA,UNH,V,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2018-07-10,0.018549,-0.004564,0.002329,0.012650,0.001371,0.003603,0.004867,0.004453,0.012699,-0.001049,...,0.004515,0.002651,0.016048,0.047606,0.025173,0.012433,0.007372,0.008633,0.014896,0.009168
2018-07-11,-0.039256,-0.002051,0.006844,-0.028416,0.000053,-0.003543,0.019656,-0.006766,-0.031899,0.000919,...,-0.006421,-0.001371,-0.022587,-0.003100,-0.008269,-0.010885,-0.003718,0.010681,-0.007798,-0.012790
2018-07-12,0.017127,0.027205,0.023715,-0.137447,0.000263,0.014553,0.020034,0.023725,0.005991,0.025635,...,0.012278,0.021671,0.014948,-0.008975,-0.003411,-0.007054,0.005734,0.012667,-0.000115,0.001211
2018-07-13,0.010668,0.014596,0.009134,-0.035813,0.005264,0.011775,-0.003712,-0.041304,-0.001770,0.004512,...,0.003670,0.011901,-0.007602,0.010401,0.005324,0.006820,0.010350,-0.003431,0.013639,0.007498
2018-07-16,-0.003932,-0.015120,0.005218,0.003902,0.005446,-0.007804,-0.004403,0.017233,-0.008546,-0.004172,...,-0.004770,-0.004932,-0.004492,0.002396,0.002774,-0.027503,-0.006649,-0.006886,-0.000684,-0.009843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,0.011259,0.010790,0.019234,0.005634,0.012140,0.012583,0.003944,0.010350,0.007104,0.007999,...,0.016294,0.016386,0.036255,0.007287,0.015799,0.016582,0.008815,0.013486,0.018797,0.005155
2023-07-03,0.002078,-0.007730,-0.001074,0.010387,0.002933,0.004941,0.001846,0.001546,-0.000890,-0.003389,...,-0.009186,-0.007488,0.002624,0.002052,0.005009,0.068954,-0.005742,0.002063,0.006489,0.001958
2023-07-05,0.019332,0.004225,0.001229,-0.016635,-0.001287,0.000166,0.010253,0.001555,-0.005725,0.017170,...,-0.005510,0.000473,-0.002263,0.005280,-0.001705,0.009506,-0.013937,0.006219,-0.000569,-0.005118


In [3]:
#Set the time period and trading days
rolling_window = 30
trading_days = 252

## Set up data for k-means model

In [4]:
# Calculate annual returns from daily returns
group_level = Y.index.year
annual_returns = Y.groupby(by=group_level).mean()
annual_returns

# Annual standard deviation
annual_stdev = annual_returns.std()*np.sqrt(trading_days)
annual_stdev.head()


ABBV     0.014605
ADBE     0.027769
AMZN     0.034284
AVGO     0.020756
BRK-B    0.004998
dtype: float64

In [5]:
# Calculate rolling 30 day returns from daily returns
thirty_day_rolling = Y.rolling(window = rolling_window).mean() 
thirty_day_rolling

# 30 day rolling standard deviation
thirty_day_stdev = thirty_day_rolling.std()*np.sqrt(rolling_window)
thirty_day_stdev.head()

ABBV     0.015850
ADBE     0.020966
AMZN     0.020776
AVGO     0.019334
BRK-B    0.012282
dtype: float64

In [6]:
# annual sharpe ratio
sharpe_ratio = (annual_returns /annual_stdev).mean()
sharpe_ratio.head()

ABBV     0.026914
ADBE     0.032127
AMZN     0.020597
AVGO     0.072943
BRK-B    0.120229
dtype: float64

In [7]:
# combine all of the measures of variablity into one dataframe
asset_quant_data = pd.concat([annual_stdev,thirty_day_stdev,sharpe_ratio], axis =1)
asset_quant_data.columns = ('annual_stdev','thirty_day_stdev','sharpe_ratio')
asset_quant_data

,annual_stdev,thirty_day_stdev,sharpe_ratio
ABBV,0.014605,0.015850,0.026914
ADBE,0.027769,0.020966,0.032127
AMZN,0.034284,0.020776,0.020597
AVGO,0.020756,0.019334,0.072943
BRK-B,0.004998,0.012282,0.120229
COST,0.015447,0.014004,0.053992
CRM,0.031064,0.021281,0.026691
CSCO,0.012570,0.015907,0.035660
CVX,0.020702,0.020654,0.016174
GOOG,0.026220,0.016592,0.030526


## k-means modeling to cluster assets

In [8]:
# using a standard scaler to scale the variability metrics to avoid bias in the model

scaled_data = StandardScaler()

asset_data_scaled = scaled_data.fit_transform(asset_quant_data[['annual_stdev','thirty_day_stdev','sharpe_ratio']])



In [9]:
# Create a DataFrame with the scaled data
asset_data_scaled = pd.DataFrame(
    asset_data_scaled,
    columns=asset_quant_data.columns
)

# Copy the asset names from the original data
asset_data_scaled["Asset_Name"] = asset_quant_data.index

# Set the asset_name column as index
asset_data_scaled = asset_data_scaled.set_index("Asset_Name")

# Display sample data
asset_data_scaled.head()

,annual_stdev,thirty_day_stdev,sharpe_ratio
Asset_Name,,,
ABBV,-0.374031,-0.226870,-0.733146
ADBE,0.323136,0.441100,-0.594913
AMZN,0.668193,0.416204,-0.900677
AVGO,-0.048304,0.227976,0.487459
BRK-B,-0.882886,-0.692596,1.741419


In [10]:
# Create a list with the number of k-values to try
# Use a range from 1 to 10
k = list(range(1,10))

In [11]:
# Create an empy list to store the inertia values
inertia = []

In [12]:
for i in k:
    k_model = KMeans(n_clusters = i, random_state = 1)
    k_model.fit(asset_data_scaled)
    inertia.append(k_model.inertia_)

In [13]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k":k, "inertia":inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

#check dataframe
elbow_df.head()

,k,inertia
0,1,84.000000
1,2,40.883194
2,3,18.108490
3,4,10.972337
4,5,7.701787


In [14]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x = "k", y = "inertia", title = "Elbow Curve", xticks = k)

:Curve   [k]   (inertia)

In [15]:
# Initialize the K-Means model using the best value for k
model = KMeans (n_clusters = 4, random_state = 1)

In [16]:
# Fit the K-Means model using the scaled data
model.fit(asset_data_scaled)

KMeans(n_clusters=4, random_state=1)

In [17]:
# Predict the clusters to group the assets using the scaled data
k = model.predict(asset_data_scaled)

# View the resulting array of cluster values.
k

array([1, 3, 3, 1, 2, 1, 3, 1, 3, 3, 3, 1, 1, 1, 2, 1, 2, 0, 1, 1, 0, 2,
       1, 0, 1, 1, 2, 3])

In [18]:
# Create a copy of the DataFrame
asset_data_scaled_predict = asset_data_scaled.copy()

In [19]:
# Add a new column to the DataFrame with the predicted clusters
asset_data_scaled_predict["clusters"] = k

# Display sample data
asset_data_scaled_predict

,annual_stdev,thirty_day_stdev,sharpe_ratio,clusters
Asset_Name,,,,
ABBV,-0.374031,-0.226870,-0.733146,1
ADBE,0.323136,0.441100,-0.594913,3
AMZN,0.668193,0.416204,-0.900677,3
AVGO,-0.048304,0.227976,0.487459,1
BRK-B,-0.882886,-0.692596,1.741419,2
COST,-0.329454,-0.467744,-0.015078,1
CRM,0.497650,0.482168,-0.739071,3
CSCO,-0.481824,-0.219348,-0.501210,1
CVX,-0.051161,0.400265,-1.017970,3


## Charts and summary of k-means

In [20]:
# Create a scatter plot using hvPlot by setting 
# `x="Sharpe Ratio"` and `y="Annual Average Returns"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the assets represented by each data point.
asset_data_scaled_predict.hvplot.scatter(
    x="annual_stdev", 
    y="sharpe_ratio",
    by = "clusters",
    hover_cols = "all").opts(yformatter="%.0f")


:NdOverlay   [clusters]
   :Scatter   [annual_stdev]   (sharpe_ratio,Asset_Name,thirty_day_stdev,clusters)

## Translating the k-means model output into segmented portfolios

In [21]:
# calculate the cumulative returns over the entire data period 
cumulative_returns = (1+Y).cumprod()-1

# select the last date in the period as the cumulative returns
# reset the index and transpose the row
asset_cumulative_returns = cumulative_returns.tail(1).T.reset_index()

# rename the columns to new data frame with the ticker as 'asset_name',
# and the cumulative returns as 'five_yr_cumulative_returns'
asset_cumulative_returns.columns = list(['asset_name','five_yr_cumulative_returns'])

#set asset name as new index
asset_cumulative_returns = asset_cumulative_returns.set_index('asset_name')

#sort the cumulative returns by higest to lowest
asset_cumulative_returns.sort_values(by = ['five_yr_cumulative_returns'], ascending = False,
                                    inplace = True)
asset_cumulative_returns

# plot the cumulative
asset_cumulative_returns.hvplot.bar(title="Cumulative Returns of Assets in Universe over the Last 5 Years").opts(xrotation=90)


:Bars   [asset_name]   (five_yr_cumulative_returns)

In [22]:
predicted_cluster_df = pd.concat([asset_data_scaled_predict,asset_cumulative_returns], axis =1).dropna()
predicted_cluster_df

,annual_stdev,thirty_day_stdev,sharpe_ratio,clusters,five_yr_cumulative_returns
ABBV,-0.374031,-0.226870,-0.733146,1,0.763311
ADBE,0.323136,0.441100,-0.594913,3,0.942867
AMZN,0.668193,0.416204,-0.900677,3,0.492565
AVGO,-0.048304,0.227976,0.487459,1,3.071375
BRK-B,-0.882886,-0.692596,1.741419,2,0.797617
COST,-0.329454,-0.467744,-0.015078,1,1.661669
CRM,0.497650,0.482168,-0.739071,3,0.478485
CSCO,-0.481824,-0.219348,-0.501210,1,0.390030
CVX,-0.051161,0.400265,-1.017970,3,0.524104
GOOG,0.241112,-0.129992,-0.637365,3,1.082059


In [23]:
predicted_cluster_df.hvplot.scatter(x = 'clusters', 
                                    y = 'five_yr_cumulative_returns',
                                    hover_cols = 'all',
                                    title="Cumulative Returns of Assets in Cluster vs. cumulative returns over the Last 5 Years")

:Scatter   [clusters]   (five_yr_cumulative_returns,index,annual_stdev,thirty_day_stdev,sharpe_ratio)

In [25]:
# Plot cumulative returns vs. stdanrd deviation to identify features within the cluster

cluster0 = predicted_cluster_df.where(predicted_cluster_df['clusters']==0).dropna()
cluster0

fig0=cluster0.hvplot.scatter(x = 'annual_stdev',
                             y = 'five_yr_cumulative_returns',
                             hover_cols = 'all',
                             title="Cumulative Returns of Assets in Cluster 0 over the Last 5 Years")


In [26]:
cluster1 = predicted_cluster_df.where(asset_data_scaled_predict['clusters']==1).dropna()
cluster1

fig1=cluster1.hvplot.scatter(x = 'annual_stdev',
                             y = 'five_yr_cumulative_returns',
                             hover_cols = 'all',
                             title="Cumulative Returns of Assets in Cluster 1 over the Last 5 Years")

In [27]:
cluster2 = predicted_cluster_df.where(asset_data_scaled_predict['clusters']==2).dropna()
cluster2

fig2=cluster2.hvplot.scatter(x = 'annual_stdev',
                             y = 'five_yr_cumulative_returns',
                             hover_cols = 'all',
                             title="Cumulative Returns of Assets in Cluster 2 over the Last 5 Years")

In [28]:
cluster3 = predicted_cluster_df.where(asset_data_scaled_predict['clusters']==3).dropna()
cluster3

fig3=cluster3.hvplot.scatter(x = 'annual_stdev',
                             y = 'five_yr_cumulative_returns',
                             hover_cols = 'all',
                             title="Cumulative Returns of Assets in Cluster 3 over the Last 5 Years")

In [29]:
#plot clusters together
fig =fig0 * fig1 * fig2 *fig3
fig

:Overlay
   .Scatter.I   :Scatter   [annual_stdev]   (five_yr_cumulative_returns,index,thirty_day_stdev,sharpe_ratio,clusters)
   .Scatter.II  :Scatter   [annual_stdev]   (five_yr_cumulative_returns,index,thirty_day_stdev,sharpe_ratio,clusters)
   .Scatter.III :Scatter   [annual_stdev]   (five_yr_cumulative_returns,index,thirty_day_stdev,sharpe_ratio,clusters)
   .Scatter.IV  :Scatter   [annual_stdev]   (five_yr_cumulative_returns,index,thirty_day_stdev,sharpe_ratio,clusters)

In [30]:
# Grouping the clusters by the 'five_yr_cumulative_returns' and the 'annual_stdev'
# this will give us our features to characterize our porfolio
cluster_characteristics = predicted_cluster_df.groupby("clusters").mean()[['five_yr_cumulative_returns', "annual_stdev"]]
cluster_characteristics
box_plot_returns = cluster_characteristics.hvplot.box(y ='five_yr_cumulative_returns',
                                                      by = 'clusters',
                                                      height=400, width=400,
                                                      title="Returns by Cluster")
box_plot_returns


:BoxWhisker   [clusters]   (five_yr_cumulative_returns)

In [31]:
# function 'label_clusters' takes our cluster characterisitcs and segments them
# based on how the features within the clusters vary from the median

def label_clusters(row):
    if row['annual_stdev'] > cluster_characteristics['annual_stdev'].median() and row['five_yr_cumulative_returns'] > cluster_characteristics['five_yr_cumulative_returns'].median():
        return 'HighRisk-HighReturn'
    elif row['annual_stdev'] < cluster_characteristics['annual_stdev'].median() and row['five_yr_cumulative_returns'] < cluster_characteristics['five_yr_cumulative_returns'].median():
        return 'LowRisk-LowReturn'
    else:
        return 'MediumRisk-MediumReturn'

cluster_characteristics['cluster_label'] = cluster_characteristics.apply(label_clusters, axis=1)

portfolio_segments = predicted_cluster_df.copy()
portfolio_segments['cluster_label'] = portfolio_segments['clusters'].map(cluster_characteristics['cluster_label'])
portfolio_segments.sort_values(by = ['cluster_label'], ascending = False, inplace = True)
portfolio_segments

,annual_stdev,thirty_day_stdev,sharpe_ratio,clusters,five_yr_cumulative_returns,cluster_label
LLY,-0.598870,-0.050504,2.781930,2,4.613404,MediumRisk-MediumReturn
CVX,-0.051161,0.400265,-1.017970,3,0.524104,MediumRisk-MediumReturn
WMT,-0.786451,-0.748715,1.114660,2,0.948769,MediumRisk-MediumReturn
PEP,-0.901405,-1.046604,1.868554,2,0.960155,MediumRisk-MediumReturn
MCD,-0.845280,-0.749922,1.951360,2,1.052375,MediumRisk-MediumReturn
ADBE,0.323136,0.441100,-0.594913,3,0.942867,MediumRisk-MediumReturn
HD,-0.156098,-0.140144,-0.851624,3,0.743269,MediumRisk-MediumReturn
GOOG,0.241112,-0.129992,-0.637365,3,1.082059,MediumRisk-MediumReturn
XOM,0.266854,0.695874,-1.075987,3,0.612031,MediumRisk-MediumReturn
CRM,0.497650,0.482168,-0.739071,3,0.478485,MediumRisk-MediumReturn


## Import the client data

In [32]:
#import json file from amazon lex 

#with open('x.json') as f:
#    data = json.load(f)
# Retrieve aggregate score
#aggregate_client_score = x['key']

In [33]:
# test score data score Medium risk and medium return
aggregate_client_score = 3

## Selecting a portfolio based on client risk profile

In [34]:
# function that is used to pick a portfolio for the client
# aggregate client score is then used to determine the
# 'LowRisk-LowReturn','HighRisk-HighReturn','MediumRisk-MediumReturn' portfolio

def get_matching_clusters(aggregate_client_score, portfolio_segments):
    if aggregate_client_score < 7:
        target_label = 'LowRisk-LowReturn'
    elif aggregate_client_score > 14:
        target_label = 'HighRisk-HighReturn'
    else:
        target_label = 'MediumRisk-MediumReturn'
    
#Filter the dataframe for the target label
    matching_cluster = portfolio_segments[portfolio_segments['cluster_label'] == target_label]
    
    return matching_cluster
    
# portfolio selected
matching_cluster = get_matching_clusters(aggregate_client_score, portfolio_segments)
matching_cluster

,annual_stdev,thirty_day_stdev,sharpe_ratio,clusters,five_yr_cumulative_returns,cluster_label
CSCO,-0.481824,-0.219348,-0.501210,1,0.390030,LowRisk-LowReturn
V,-0.596619,-0.597435,0.123768,1,0.804007,LowRisk-LowReturn
UNH,-0.394566,-0.482102,-0.482904,1,0.959350,LowRisk-LowReturn
PG,-0.560729,-0.850580,0.305032,1,1.180418,LowRisk-LowReturn
AVGO,-0.048304,0.227976,0.487459,1,3.071375,LowRisk-LowReturn
MSFT,0.037270,-0.474338,0.025121,1,2.500693,LowRisk-LowReturn
MRK,-0.420346,-0.761707,0.028811,1,1.145971,LowRisk-LowReturn
MA,-0.451017,-0.289677,-0.061857,1,0.988246,LowRisk-LowReturn
COST,-0.329454,-0.467744,-0.015078,1,1.661669,LowRisk-LowReturn
KO,-0.807801,-0.802926,0.248756,1,0.574142,LowRisk-LowReturn


In [35]:
# creating a list of assets that will be used to build the porfolio
targeted_cluster_list = []
targeted_cluster_list = matching_cluster.index.tolist()
targeted_cluster_list

['CSCO',
 'V',
 'UNH',
 'PG',
 'AVGO',
 'MSFT',
 'MRK',
 'MA',
 'COST',
 'KO',
 'JPM',
 'JNJ',
 'ABBV']

## Portfolio Optimization

*Optimal Weighted Portfolio on Cluster 2*

In [36]:
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*5) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

cluster_stock_data = yf.download(targeted_cluster_list, start = start, end = end)
adj_close_prices = cluster_stock_data.loc[:,('Adj Close', slice(None))]
adj_close_prices.columns = targeted_cluster_list

display(start)
display(end)
display(adj_close_prices)

[*********************100%***********************]  13 of 13 completed


'2018-07-09'

'2023-07-08'

,CSCO,V,UNH,PG,AVGO,MSFT,MRK,MA,COST,KO,JPM,JNJ,ABBV
Date,,,,,,,,,,,,,
2018-07-09,76.844055,207.976456,197.263443,36.682663,110.339226,92.217522,37.963547,195.488907,50.792839,96.329506,68.243790,235.578003,131.069366
2018-07-10,78.269432,210.607361,197.974182,36.846004,111.503441,91.650169,38.450912,196.062897,51.022152,96.584862,69.961723,237.314636,132.200882
2018-07-11,75.196960,204.622665,197.272781,36.596691,110.505524,91.452492,38.297012,198.320099,50.694572,96.452454,69.383240,236.432449,133.612976
2018-07-12,76.484917,176.497955,200.143723,37.464970,111.836082,91.847893,38.194397,202.153412,51.316990,98.542671,69.146584,237.788284,135.305466
2018-07-13,77.300858,170.177078,202.500320,35.917542,110.234161,91.426697,38.254257,200.334061,51.505344,99.715469,69.514709,240.249298,134.841263
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,134.729996,867.429993,538.380005,51.350605,165.520004,144.447983,60.220001,392.732056,115.389999,340.540009,151.740005,480.640015,237.479996
2023-07-03,135.009995,876.440002,541.039978,51.430000,163.350006,145.610001,60.580002,393.391083,114.330002,337.989990,152.500000,477.880005,237.970001
2023-07-05,137.619995,861.859985,541.130005,51.509998,162.809998,144.639999,61.029999,394.160004,113.699997,338.149994,152.240005,471.220001,239.449997


In [37]:
mu = mean_historical_return(adj_close_prices)
S = CovarianceShrinkage(adj_close_prices).ledoit_wolf()

ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.1)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
cleaned_weights

OrderedDict([('CSCO', 0.01929),
             ('V', 0.24644),
             ('UNH', 0.19997),
             ('PG', 0.0),
             ('AVGO', 0.0),
             ('MSFT', 0.0),
             ('MRK', 0.0),
             ('MA', 0.0),
             ('COST', 0.15756),
             ('KO', 0.22369),
             ('JPM', 0.15304),
             ('JNJ', 0.0),
             ('ABBV', 0.0)])

In [38]:
ef.portfolio_performance(verbose=True)

Expected annual return: 24.2%
Annual volatility: 21.8%
Sharpe Ratio: 1.02


(0.24162311676432371, 0.21820694763697693, 1.0156556386693523)

In [39]:
latest_prices = get_latest_prices(adj_close_prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=100000) #total_portfolio_value=100000 CHANGE THIS TO OUTPUT FROM LEX ANSWER
allocation, leftover = da.lp_portfolio()
allocation

{'CSCO': 14, 'V': 29, 'UNH': 38, 'COST': 145, 'KO': 66, 'JPM': 103}

In [40]:
allocation_weights = []
for key in allocation:
    allocation_weights.append(allocation[key])

total_shares = sum(allocation_weights)
total_shares

allocation_weights = [x / total_shares for x in allocation_weights]
allocation_weights  

allocated_tickers = list(allocation.keys())
allocated_tickers

['CSCO', 'V', 'UNH', 'COST', 'KO', 'JPM']

In [41]:
dfs = []

for ticker in allocated_tickers:
    data = yf.download(ticker, start, interval='1d')
    open_df = pd.DataFrame(data['Open']).rename(columns={'Open': 'open'})
    adj_close_df = pd.DataFrame(data['Adj Close']).rename(columns={'Adj Close': 'close'})
    daily_return_df = pd.DataFrame(data['Adj Close'].pct_change()).rename(columns={'Adj Close': 'daily_return'})
    dfs.append(pd.concat([open_df, adj_close_df, daily_return_df], axis=1, keys=[ticker]*3))

allocated_data = pd.concat(dfs, axis=1).dropna()
allocated_data 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


CSCO                                   V              \
                 open      close daily_return        open       close   
Date                                                                    
2018-07-10  42.869999  36.846004     0.004453  135.570007  132.200928   
2018-07-11  42.560001  36.596703    -0.006766  136.630005  133.612946   
2018-07-12  42.759998  37.464981     0.023726  139.009995  135.305511   
2018-07-13  43.520000  35.917542    -0.041304  139.789993  134.841248   
2018-07-16  42.049999  36.536518     0.017233  139.419998  133.912796   
...               ...        ...          ...         ...         ...   
2023-06-30  51.880001  51.350605     0.010350  236.270004  237.479996   
2023-07-03  51.360001  51.430000     0.001546  237.000000  237.970001   
2023-07-05  50.970001  51.509998     0.001555  234.410004  239.449997   
2023-07-06  51.070000  51.110001    -0.007765  238.270004  238.880005   
2023-07-07  51.000000  50.990002    -0.002348  236.729996  236.449997   

                                UNH                                 COST  \
           daily_return        open       close daily_return        open   
Date                                                                       
2018-07-10     0.008634  254.179993  237.314636     0.007372  210.990005   
2018-07-11     0.010681  255.139999  236.432404    -0.003718  211.210007   
2018-07-12     0.012668  256.179993  237.788300     0.005735  212.539993   
2018-07-13    -0.003431  256.390015  240.249329     0.010350  214.000000   
2018-07-16    -0.006886  258.290009  238.651962    -0.006649  216.639999   
...                 ...         ...         ...          ...         ...   
2023-06-30     0.013486  478.000000  480.640015     0.008815  534.169983   
2023-07-03     0.002063  478.100006  477.880005    -0.005742  537.250000   
2023-07-05     0.006219  476.029999  471.220001    -0.013937  541.219971   
2023-07-06    -0.002380  469.350006  469.359985    -0.003947  538.880005   
2023-07-07    -0.010173  465.000000  461.579987    -0.016576  533.890015   

                                            KO                          \
                 close daily_return       open      close daily_return   
Date                                                                     
2018-07-10  197.974197     0.003603  44.509998  38.450920     0.012838   
2018-07-11  197.272781    -0.003543  44.990002  38.297009    -0.004003   
2018-07-12  200.143738     0.014553  44.830002  38.194401    -0.002679   
2018-07-13  202.500320     0.011774  44.700001  38.254261     0.001567   
2018-07-16  200.919968    -0.007804  44.730000  38.245705    -0.000224   
...                ...          ...        ...        ...          ...   
2023-06-30  538.380005     0.012583  60.000000  60.220001     0.003332   
2023-07-03  541.039978     0.004941  60.200001  60.580002     0.005978   
2023-07-05  541.130005     0.000166  60.279999  61.029999     0.007428   
2023-07-06  537.369995    -0.006948  60.730000  60.560001    -0.007701   
2023-07-07  525.049988    -0.022926  60.310001  59.759998    -0.013210   

                   JPM                           
                  open       close daily_return  
Date                                             
2018-07-10  108.019997   91.650177    -0.006152  
2018-07-11  106.309998   91.452469    -0.002157  
2018-07-12  107.419998   91.847885     0.004324  
2018-07-13  107.300003   91.426682    -0.004586  
2018-07-16  107.050003   95.054192     0.039677  
...                ...         ...          ...  
2023-06-30  144.600006  144.447983     0.014014  
2023-07-03  146.190002  145.610001     0.008045  
2023-07-05  144.940002  144.639999    -0.006662  
2023-07-06  143.020004  143.210007    -0.009887  
2023-07-07  143.910004  144.339996     0.007890  

[1257 rows x 18 columns]

In [42]:
allocated_closing_prices = adj_close_prices[allocated_tickers]
allocated_closing_prices

,CSCO,V,UNH,COST,KO,JPM
Date,,,,,,
2018-07-09,76.844055,207.976456,197.263443,50.792839,96.329506,68.243790
2018-07-10,78.269432,210.607361,197.974182,51.022152,96.584862,69.961723
2018-07-11,75.196960,204.622665,197.272781,50.694572,96.452454,69.383240
2018-07-12,76.484917,176.497955,200.143723,51.316990,98.542671,69.146584
2018-07-13,77.300858,170.177078,202.500320,51.505344,99.715469,69.514709
...,...,...,...,...,...,...
2023-06-30,134.729996,867.429993,538.380005,115.389999,340.540009,151.740005
2023-07-03,135.009995,876.440002,541.039978,114.330002,337.989990,152.500000
2023-07-05,137.619995,861.859985,541.130005,113.699997,338.149994,152.240005


In [43]:
investment_timeframe = 5
monte_carlo_simulation = MCSimulation(
    portfolio_data= allocated_data,
    weights=allocation_weights,
    num_simulation=500,
    num_trading_days=252*investment_timeframe,
)

monte_carlo_simulation.portfolio_data.head()

NameError: name 'MCSimulation' is not defined

In [ ]:
monte_carlo_simulation.calc_cumulative_return()

In [ ]:
monte_carlo_simulation.plot_simulation()

In [ ]:
monte_carlo_simulation.plot_distribution()

In [ ]:
monte_carlo_simulation_statistics = monte_carlo_simulation.summarize_cumulative_return()
monte_carlo_simulation_statistics

In [ ]:
ci_lower_ten_cumulative_return = monte_carlo_simulation_statistics[8]
ci_upper_ten_cumulative_return = monte_carlo_simulation_statistics[9]

In [ ]:
print(f"There is a 95% chance that the portfolio will provide returns within in the range of"
      f" {ci_lower_ten_cumulative_return: .2f} and {ci_upper_ten_cumulative_return: .2f} percent"
      f" over {investment_timeframe: .0f} years")